## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info() #общая информация о таблице,из которой можно увидеть количество строк, количество отсутствующих данных, тип данных
df.head() # 5 первых строк, чтобы получить визуальное представление о таблице
df.sort_values('children',ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19562,20,-721.344373,30,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,133319.593450,сыграть свадьбу
11715,20,-1034.170229,43,среднее,1,женат / замужем,0,M,сотрудник,0,220771.388727,свой автомобиль
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
5315,20,-2047.754733,24,среднее,1,женат / замужем,0,F,сотрудник,0,100415.236833,покупка коммерческой недвижимости
13489,20,385267.263676,62,среднее,1,гражданский брак,1,F,пенсионер,0,192221.076774,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
16265,-1,-2802.218127,40,высшее,0,женат / замужем,0,M,сотрудник,0,111984.472021,покупка жилья
6013,-1,-1361.258696,46,высшее,0,женат / замужем,0,F,сотрудник,0,143008.454914,строительство собственной недвижимости
21140,-1,-1422.668059,44,среднее,1,женат / замужем,0,F,компаньон,0,169562.091999,операции со своей недвижимостью
19100,-1,-617.246968,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,122205.497527,строительство жилой недвижимости


##### Описание данных

| наименование      | описание                         | 
|:----------------- |:---------------------------------| 
| children          | количество детей в семье         |     
| days_employed     | трудовой стаж в днях             |
| dob_days          | возраст клиента в годах          |
| education         | образование клиента              |
| education_id      | идентификатор образования        |
| family_status     | семейное положение               |
| family_status_id  | идентификатор семейного положения|
| gender            | пол клиента                      |
| income_type       | тип занятости                    |
| debt              | имел ли задолжность по кредиту   |
| total_income      | доход в месяц                    |
| purpose           | цель получения кредита           |


### Вывод

Во время изучения общей информации были обнаружены пропущенные значения в days_employed, total_income.
Возможные причины имеют технический характер, так как количество и местоположение NaN в days_employed, total_income совпадает.
Смею предположить,что клиент не предоставил информацию о трудовом стаже и доходе, что несколько странно.Можно было бы списать на человеческий фактор, в том смысле,что стаж и доход относятся к трудовой деятельности и могли бы содержаться в одной сводной таблице, которая была утеряна, но это только ничем не обоснованные предположения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
pd.isnull(df).sum() #количество отсутствующих данных

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверил комментарии на тему отсутствия обработки пропусков. Вывел отдельной строкой с выводами где и чем заполняются пропуски,перезапустил Kernel, дважды. Все работает. Возможно, что-то упустил или не понял.Жду новые комментарии. Возможно это произошло по той причине, что после отправки задания на проверку вместо традиционного сообщения было получено  это - proficiency.content.jupyterHomework.reviewStatus.new 
Так же я перед отправкой не перезапустил Kernel, хотя это лишь мои предположения.

In [3]:
df['days_employed'] = df['days_employed'].abs() #замена типа данных float на тип данных int

In [4]:
days_employed_median = df.days_employed.median() #переменная медиана data_employed

In [5]:
total_income_median = df.total_income.median() #переменная медиана total_income

In [6]:
df['total_income'] = df['total_income'].fillna(value = total_income_median) 

*Заполняем пропущенные значения медианой для total_income*

In [7]:
df['days_employed'] = df['days_employed'].fillna(value = days_employed_median) 

*Заполняем пропущенные значения медианой для days_employed*

In [8]:
pd.isnull(df).sum() # перепроверяем наличие Nan в таблице

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [9]:
df.total_income.value_counts().head() 
#смотрим действительно ли незаполненные значения df.total_income заменились на медиану

145017.937533    2175
112874.418757       1
104381.857170       1
182036.676828       1
122421.963500       1
Name: total_income, dtype: int64

In [10]:
df.days_employed.value_counts().head()
#смотрим действительно ли незаполненные значения df.days_employed заменились на медиану

2194.220567    2175
986.927316        1
1893.222792       1
4236.274243       1
6620.396473       1
Name: days_employed, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропущенные значения отсутствуют.

Медиана не чувствительна к выбросам в отличие от средне арифметического значения.
Вызвав метод describe() можно увидеть, что и в столбце df.total_income и в df.days_employed среднее значение (mean) и медиана (50%) стоят на некоротом расстоянии друг от друга, исходя из этого можно сделать вывод,что данных столбцах есть выброс. 

In [12]:
df.total_income.describe()

count    2.152500e+04
mean     1.651595e+05
std      9.786607e+04
min      2.066726e+04
25%      1.077982e+05
50%      1.450179e+05
75%      1.955436e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [13]:
df.loc[df['purpose'] == 'образование'].sort_values('dob_years', ascending=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6680,0,351171.863199,71,среднее,1,Не женат / не замужем,4,F,пенсионер,0,97259.954125,образование
11261,0,339187.226615,70,среднее,1,женат / замужем,0,F,пенсионер,0,30075.645693,образование
7262,0,369739.388681,70,среднее,1,вдовец / вдова,2,F,пенсионер,0,71943.614288,образование
14028,0,338712.950062,69,среднее,1,вдовец / вдова,2,F,пенсионер,0,134549.237746,образование
6602,0,367668.223702,69,начальное,3,в разводе,3,F,пенсионер,0,49034.074944,образование
...,...,...,...,...,...,...,...,...,...,...,...,...
4219,0,1127.900955,21,неоконченное высшее,2,женат / замужем,0,M,компаньон,0,296073.622355,образование
13022,0,208.074390,21,среднее,1,Не женат / не замужем,4,M,сотрудник,0,89414.796891,образование
18732,2,986.789230,0,высшее,0,гражданский брак,1,F,компаньон,0,287328.538328,образование
7034,0,366067.781030,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,263121.074528,образование


In [14]:
df.days_employed.describe()

count     21525.000000
mean      60378.032733
std      133257.558514
min          24.141633
25%        1025.608174
50%        2194.220567
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

### Вывод

Отсутствующие данные объектов заменил на нулевые значения, для этого использовал метод fillna().Решение было принято с тем смыслом, что удалять 10% информации расточительно.

### Замена типа данных

In [15]:
df['days_employed'] = df['days_employed'].astype('int') #замена типа данных float на тип данных int
df['children'] = df['children'].abs()#отрицательных значений в количестве детей не бывает
df['children'] = df.replace(20,2) # 20 детей это перебор,особенно когда девушке всего лишь 24 года клиент №5315

In [16]:
df['children'].value_counts() # проверяем нет ли отрицательных значений и ошибочных данных

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

Так как колонка days_employed это не что иное как трудовой стаж в днях, а стаж измеряемый в днях не может быть дробными и тем более отрицательным.
Количество детей не может быть отрицательным и у одного клиента 20 детей, тоже маловерятно 

### Обработка дубликатов

In [17]:
df['education'] = df['education'].str.lower() #перевел education к нижнему регистру

In [18]:
df[['education_id' , 'education']][df.duplicated(keep=False)]
# поиск дубликатов в логически связанных полях: образование / идентификатор образования

,education_id,education
120,1,среднее
520,1,среднее
541,1,среднее
554,1,среднее
680,0,высшее
...,...,...
20702,1,среднее
21032,1,среднее
21132,1,среднее
21281,0,высшее


In [19]:
df[['family_status_id' , 'family_status']][df.duplicated(keep=False)]
# поиск дубликатов в логически связанных полях: семейное положение / идентификатор семейного положения

,family_status_id,family_status
120,0,женат / замужем
520,1,гражданский брак
541,0,женат / замужем
554,0,женат / замужем
680,0,женат / замужем
...,...,...
20702,0,женат / замужем
21032,0,женат / замужем
21132,0,женат / замужем
21281,0,женат / замужем


In [20]:
df = df.drop_duplicates() # удаление дубликатов из таблицы

### Вывод

Для поиска дубликатов использовался метод duplicated() с поиском по всем колонкам, для удаление дубликатов использовался метод drop_duplicated() и обновил нумерацию индексов при помощи reset_index(drop_True).
Возможная причина появления дубликатов в данных человеческий фактор, ошибка заполнения таблицы.

### Лемматизация

In [21]:
from pymystem3 import Mystem # импорт библиотеки pymystem3. Спасибо Ильи Валентиновичу за создание морфологического алгоритма
m = Mystem()
from collections import Counter
lemmas = []                   # задаем цикл по row['purpose'] с целью собрать список лемм
for index,row in df.iterrows():
    lemmas.append(' '.join(m.lemmatize(row['purpose'])))
         

In [22]:
df['lemmas'] = lemmas # получившийся список загружаем в колонку df.lemmas

In [23]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,сыграть свадьба \n


### Вывод

Лемматизацию целей кредита производилась с использованием библиотеки pymystem3
В качестве "словарей" было выделено 4 категории: недвижимость, автомобиль, образование, свадьба. Такое решение было принято из 
логических соображений, так как эти категории в совокупности целиком описывают цели кредита.

### Категоризация данных

In [24]:
def get_purpose(lemmas):
    """
    Функция для приведения списка лемм 
    к одной из четырёх категорий:
        свадьба
        автомобиль
        недвижимость
        образование
    """
    if 'свадьба'  in lemmas:
        return 'свадьба'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'жилье' in lemmas:
        return 'недвижимость'
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'образование' in lemmas:
        return 'образование'

In [25]:
df['purpose'] = df['lemmas'].apply(get_purpose) # приведение df.purpose к одной из четырех категорий

In [26]:
df['purpose'].value_counts() # посмотрим что получилось

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

In [27]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба,сыграть свадьба \n


### Вывод

Категоризацию данных производил продолжая тему лемматизации. В качестве вывода могу только повторить, что цели кредита свел к 4 логическим категориям, может способ решения не очень экономичный, наверняка есть способ преобразования более быстрый.

### Шаг 3. Ответьте на вопросы

- #### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
df_children = df.pivot_table(index='children', columns='debt', values='total_income', aggfunc='count')
# формируем сводную таблицу:
# колонки - имел ли задолжность по возврату кредитов. 
# индексы - количество детей в семье
# значения - доход в месяц
# функцией агрегирования будет count (подсчёт количества записей)
df_children 

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [29]:
df_children['prob_%'] = df_children[0] / (df_children[0] + df_children[1]) * 100
# создаем новую колонку в ней высчитываем процент клиентов  не имеющих задолжности по возврату кредита

In [30]:
df_children = df_children.fillna(0) # заполняем пропущенные значения

In [31]:
df_children # выводим сводную таблицу на экран

debt,0,1,prob_%
children,,,
0,13028.0,1063.0,92.456178
1,4410.0,445.0,90.834192
2,1926.0,202.0,90.507519
3,303.0,27.0,91.818182
4,37.0,4.0,90.243902
5,9.0,0.0,0.000000


Спасибо, за изящное решение с помощью сводной таблицы.
В ней все гораздо наглядней и понятней

### Вывод

Зависимость между наличием детей и возвратом кредита в срок есть. Лишь 33% клиентов из числа,не имеющие задолжностей по возврату кредита, с детьми (от 1 до 5 )в состоянии вовремя вернуть кредит. Думаю,часть денег уходит на подрастающее поколение.
Хотя разность в плажеспособности клиенов-родителей и клиенов без детей не такая и большая 90,8% и 92,5%.
Можно сделать вывод,что хоть зависимость и есть между наличием детей и возвратом кредита в срок, в процентном соотношении она не большая, всего 1,7%

- #### "Есть ли зависимость между семейным положением и возвратом кредита в срок?"

In [32]:
df_family_status = df.pivot_table(index='family_status', columns='debt', values='total_income', aggfunc='count')
# формируем сводную таблицу:
# колонки - имел ли задолжность по возврату кредитов. 
# индексы - семейное положение
# значения - доход в месяц
# функцией агрегирования будет count (подсчёт количества записей)

In [33]:
df_family_status['prob_%'] = df_family_status[0] / (df_family_status[0] + df_family_status[1]) * 100
# создаем новую колонку в ней высчитываем процент клиентов  не имеющих задолжности по возврату кредита

In [34]:
df_family_status # выводим сводную таблицу на экран

debt,0,1,prob_%
family_status,,,
Не женат / не замужем,2536,274,90.249110
в разводе,1110,85,92.887029
вдовец / вдова,896,63,93.430657
гражданский брак,3763,388,90.652855
женат / замужем,11408,931,92.454818


### Вывод

"Есть ли зависимость между семейным положением и возвратом кредита в срок?"
Однозначно на этот вопрос ответить сложно, если рассматривать в разрезе женатые / не женатые то да.Процент клиентов без задолжности состоящих в браке выше чем не женатых и в тоже время процент клиентов потерявшие своих супругов,но вовращающие кредит вовремя выше в других категориях, и те клиенты, что вышли из отношений тоже не уступают клиентам в них состоящих 92,88%.


- #### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
import numpy as np  # импорт библиотеки numpy

In [36]:
class_income = []  # список будущего класса дохода

In [37]:
total_income_25 = np.percentile(df.total_income,25) #переменная первая квантиль

In [38]:
total_income_50 = np.percentile(df.total_income,50) #переменная вторая квантиль

In [39]:
total_income_75 = np.percentile(df.total_income,75) #переменная третья квантиль

In [40]:
"""
Цикл распределяет значения столбца дохода в месяц по категорям от 1 до 4 по квартилям
"""

for i in df['total_income']:
    if i >= total_income_50:
        if i > total_income_75:
            class_income.append(4)
        else:
            class_income.append(3)
    if i < total_income_50:
        if i < total_income_25:
            class_income.append(1)
        else:
            class_income.append(2)

In [41]:
df['class_income'] = class_income #заносим получившийся список в колонку класс дохода

In [42]:
df_class_income = df.pivot_table(index='class_income', columns='debt', values='total_income', aggfunc='count')
# формируем сводную таблицу:
# колонки - имел ли задолжность по возврату кредитов. 
# индексы - класс дохода
# значения - доход в месяц
# функцией агрегирования будет count (подсчёт количества записей)

In [43]:
df_class_income['prob_%'] = df_class_income[0] / (df_class_income[0] + df_class_income[1]) * 100
# создаем новую колонку в ней высчитываем процент клиентов  не имеющих задолжности по возврату кредита

In [44]:
df_class_income # выводим сводную таблицу на экран

debt,0,1,prob_%
class_income,,,
1,4937,427,92.039523
2,3934,377,91.254929
3,5861,554,91.363991
4,4981,383,92.859806


### Вывод

 Исходя из полученных данных можно сделать вывод, что зависимость между уровнем дохода и возвратом кредита в срок нет.
 Клиентов с высоким уровнем дохода возвращающих кредит в срок примерно столько же сколько и с низким. Нормальное распределение не имеет ярко выраженного смещения ни в сторону низких доходов ни в сторону высоких.

- #### Как разные цели кредита влияют на его возврат в срок?

In [45]:
df_purpose = df.pivot_table(index='purpose', columns='debt', values='total_income', aggfunc='count')
# формируем сводную таблицу:
# колонки - имел ли задолжность по возврату кредитов. 
# индексы - цель кредита
# значения - доход в месяц
# функцией агрегирования будет count (подсчёт количества записей)

In [46]:
df_purpose['prob_%'] = df_purpose[0] / (df_purpose[0] + df_purpose[1]) * 100
# создаем новую колонку в ней высчитываем процент клиентов  не имеющих задолжности по возврату кредита

In [47]:
df_purpose # выводим сводную таблицу на экран

debt,0,1,prob_%
purpose,,,
автомобиль,3903,403,90.640966
недвижимость,10029,782,92.766627
образование,3643,370,90.779965
свадьба,2138,186,91.996558


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 14 columns):
children            21454 non-null object
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
lemmas              21454 non-null object
class_income        21454 non-null int64
dtypes: float64(1), int64(6), object(7)
memory usage: 2.5+ MB


### Вывод

Процент успешного возврата кредита в срок целью которого является:
    Недвижимость 92,76%
    Автомобиль   90,64%
    Образование  90,77%
    Свадьба      91,99%
ВЫВОД: Разные цели влияют на успешный возврат кредита в срок    

### Шаг 4. Общий вывод

Согласно полученным данным, можно сделать заключение, на возврат кредита в срок: 

    - зависимость от количества детей(клиенты с детьми  92,5%, клиенты без детей 90,8%) есть
    
    - семейные положение так же оказывает влияние:
        женат / замужем          92,47%
        гражданский брак         90,71%
        Не женат / не замужем    90,25%
        в разводе                92,88%
        вдовец / вдова           93,43%
        
    - ярко выраженной зависимости между уровнем дохода и возвратом в срок не обнаружено
       
    - цель определенно оказывает влияние:
        Недвижимость 92,78%
        Автомобиль   90,66%
        Образование  90,80%
        Свадьба      92,07%
      

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.